## Top Down Modeling Process

The original `Reconstruction` search feature begins the search space with an empty graph, building up the body from `NewBodyFeatureOperation`, `JoinFeatureOperation`, `CutFeatureOperation`, or `IntersectFeatureOperation`. The search space is originally restricted to `NewBodyFeatureOperation` since the other operations are not possible on an empty graph. 

<p float="left"> 
  <img src="images/best_search_1.png" style="display:inline;margin:1px;height:300px" /> 
  <img src="images/best_search_2.png" style="display:inline;margin:1px;height:300px" />  
  <img src="images/best_search_3.png" style="display:inline;margin:1px;height:300px" /> 
</p>

<center> Example face extrusion process for original best_search process </center>

### Simulating manufacturing processes in design

#### Overview

There are 2 umbrellas of manufacturing for manufactured parts, additive manufacturing and subtractive manufacturing. Additive manufacturing, such as injection molding and 3D printing, *add* material to create a solid object. Opposite of this is machining, which starts with a block of materials and *subtracts* material to create the final design. Additive manufacturing is like working with clay, whereas subtractive manufacturing is like working with a granite block and chisel. 

Good design practices recommend designing your part in the way that it will be manufactured. If a part is to be manufactured via a subtractive manufacturing process, it is best practice to start your model as a block of material and cut away material. This allows the designer to visualize and optimize the design for the manufcaturing process and allows for easier editing of the design later on. 

The search feature implemented in the $Search$ class and its children start with a blank graph and *add* bodies to form the design. Though its possible for the agent to choose to extrude a bounding box then cut away to the target, it is not guaranteed, and typically not what happens. Further, due to the limitations imposed by the message passing network, namely the fact it returns 2 faces and an operation, as well the rules imposed on the agent (namely the faces must be parallel) there are several more complex shapes that it completely fails at making. Essentially, if there is a face with no parallel faces it canot be created. See examples below of a shape the agent fails to reconstruct. 

<p float="center"> 
  <img src="images/77211_d46ae17d_0013.png" style="height:300px"/> 
</p>

<center> Pyramidal face with no parallel faces fails to be reconstructed </center>

Looking into the ground truth of the above model, it is apparent that the final model was formed from a bounding box, then two extrude cut operations were used to complete the shape. Since the final cuts go all the way through the model, there are no parallel faces for the agent to complete a face extrusion. 

<p float="center"> 
  <img src="images/77211_d46ae17d_0013_0112.png" style="display:inline;margin:1px;height:300px" /> 
  <img src="images/77211_d46ae17d_0013_0116.png" style="display:inline;margin:1px;height:300px" />  
  <img src="images/77211_d46ae17d_0013.png" style="display:inline;margin:1px;height:300px" /> 
</p>

<center> Extrusion steps to recreate the solid object </center>

This particular model fails the search algorithm due to the parallel face limitation of face extrusions. If we were able to start with the bounding box and cut away to reveal the model, however, this could be reconstructed using the existing framework of the search algorithm.

#### Implementation

To implement this, we needed to modify the search algorithm to add a bounding box before each face extrusion operation. The code below was added to the $search\_best$ script used to search and update action probabilities based on the step or steps with best $IoU$. 

```python
    def search_bounding_box(self, agent, budget, max_point, min_point, score_function=None, screenshot=False):
        super().search_bounding_box(agent, budget, max_point, min_point, score_function, screenshot)
        # the length of rollout is the same as the number of planar faces as a maximum
        rollout_length = 0
        for node in self.target_graph["nodes"]:
            if node["surface_type"] == "PlaneSurfaceType":
                rollout_length += 1
        if rollout_length < 2:
            # There exist some designs with no planar faces that we can't handle
            # We need at least 2 faces
            raise Exception("Not enough valid planar faces in target")

        used_budget = 0
        max_score = 0
        max_scores = []

        fringe = PriorityQueue()
        fringe.put(PriorityAction(0, ()))

        # while there is item in the fridge and we still have budget
        while fringe.qsize() > 0 and used_budget < budget:
            # Revert environment to target and set current graph to bounding box

            ###################################################################
            # Instead of starting with a blank graph and adding, we start with
            # a bounding box and subtract from it
            self.env.revert_to_target()
            cur_graph = self.env.get_bounding_graph(min_point, max_point)
            ###################################################################
            
            priority_action = fringe.get()
            # nll is something like 10, prefix is something like (a1, a4, a10)
            nll = priority_action.nll
            prefix = priority_action.prefix
            new_graph, cur_iou = self.env.extrudes(list(prefix), revert=False)

            if len(prefix) > 0:
                used_budget += 1
                take_screenshot = screenshot
                if cur_iou is not None:
                    max_score = max(max_score, cur_iou)
                else:
                    # We only want to take screenshots when something changes
                    take_screenshot = False
                if new_graph is not None:
                    cur_graph = new_graph

                log_data = {
                    # "rollout_attempt": rollout_attempt,
                    # "rollout_step": i,
                    # "rollout_length": rollout_length,
                    "used_budget": used_budget,
                    "budget": budget,
                    "current_iou": cur_iou,
                    "max_iou": max_score,
                    "prefix": list(prefix)
                }
                self.log.log(log_data, take_screenshot)
                max_scores.append(max_score)
                # Stop early if we find a solution
                if math.isclose(max_score, 1, abs_tol=0.00001):
                    return max_scores
                # Stop if the rollout hits the budget
                if used_budget >= budget:
                    break
            # If there was an invalid operation
            # continue without adding it to the search space
            if (new_graph is None or cur_iou is None) and len(prefix) > 0:
                continue

            # extend the current prefix by 1 step forward
            actions, action_probabilities = agent.get_actions_probabilities(cur_graph, self.target_graph)
            # Filter for clearly bad actions
            action_probabilities = self.filter_bad_actions_bounded(cur_graph, actions, action_probabilities)
            # Convert probability to logpr so they can be added rather than multiplied for numerical stability
            action_logprs = np.log(action_probabilities)
            # add to the candidates back to fringe
            for (a, a_logpr) in zip(actions, action_logprs):
                child_prefix = prefix + (a,)
                child_nll = nll - a_logpr
                # do not add a prefix that's longer than rollout length
                if len(child_prefix) < rollout_length:
                    fringe.put(PriorityAction(child_nll, child_prefix))

            print(f"[{used_budget}/{budget}] Score: {max_score}")
        return max_scores
```

We call the Fusion 360 environment and the custom function, `get_bounding_graph` which does the following:

```python
    def get_bounding_graph(self, min_point, max_point):
        """Wrapper to extrude a bounding rectangle around a solid model"""
        _min_point = min_point
        _max_point = max_point

        start_face, end_face = self.boundary_points(_min_point, _max_point)
        distance = _max_point[0] - _min_point[0]

        # Create a new sketch vias the server client
        r = self.client.add_sketch("XY")
        response_json = r.json()
        sketch_name = response_json["data"]["sketch_name"]

        # Add points to new sketch
        for i, point in enumerate(start_face):
            r = self.client.add_point(sketch_name, 
                {"x": point[0], "y": point[1], "z":point[2]})
            response_json = r.json()

        # Close profile of points to make a face to extrude
        r = self.client.close_profile(sketch_name)
        response_json = r.json()
        profile_id = (list(response_json['data']['profiles'].keys())[0])

        # Extrude by distance between start and end face
        r = self.client.add_extrude(sketch_name, profile_id, -1 * distance, "NewBodyFeatureOperation")

        # Convert extrusion to graph and return graph format
        r = self.client.graph(file='', dir='', format='PerFace')
        response_json = r.json()
        return response_json['data']['graph']
```
```python
def boundary_points(self, min_point, max_point):
        """Return points needed to make a boundary extrude encompassing model"""
        start_face = []
        end_face = []
        start_face.append(min_point)
        start_face.append((min_point[0], min_point[1], max_point[2]))
        start_face.append((min_point[0], max_point[1], max_point[2]))
        start_face.append((min_point[0], max_point[1], min_point[2]))

        end_face.append((max_point[0], min_point[1], min_point[2]))
        end_face.append((max_point[0], min_point[1], max_point[2]))
        end_face.append((max_point[0], max_point[1], max_point[2]))
        end_face.append((max_point[0], max_point[1], min_point[2]))

        return start_face, end_face
```

This creates a bounding box around the target object. Then, through the function `filter_bad_actions_bounded` we force the probabilities to near zero of any operation that is not a `CutFeatureOperation` operation as shown in the code snippet below:

<p float="center"> 
  <img src="images/bounding_box.gif" style="height:200px"/> 
</p>

<center> GIF showing the bounding box extrusion step </center>

```python
def filter_bad_actions_bounded(self, current_graph, actions, action_probabilities):
        """Filter out some actions we clearly don't want to take"""
        assert self.target_graph is not None
        epsilon = 0.00000000001
        # Adjust the probabilities of bad actions
        for index, action in enumerate(actions):
            if action["operation"] == "NewComponentFeatureOperation":
                action_probabilities[index] = epsilon
            if action["operation"] != "CutFeatureOperation":
                action_probabilities[index] = epsilon
            if action_probabilities[index] < epsilon:
                action_probabilities[index] = epsilon

        action_probabilities = action_probabilities / sum(action_probabilities)
        return action_probabilities
```

### Results and Analysis

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

import sys
import os
from IPython.core.display import display, HTML, Image

In [88]:
control_log = "../tools/search/log/"
bounded_log = "../tools/search/log_bounded/"

In [89]:
control_directories = os.listdir(path=control_log)
bounded_directories = os.listdir(path=bounded_log)

In [90]:
dict_df = {'uuid': [],
             'used_budget': [],
             'max_iou': [],
             'exact': [],
             'bounded': []}

In [91]:
bounded = False
for directory in control_directories:
    # Return all files within directory
    try:
        files = os.listdir(os.path.join(control_log, directory))
        for file in files:
            if file.endswith('.json'):
                with open(os.path.join(control_log, directory, file), 'r') as f:
                    f = json.load(f)
                    dict_df['uuid'].append(directory)
                    dict_df['used_budget'].append(f[-1]['used_budget'])
                    dict_df['max_iou'].append(f[-1]['max_iou'])
                    dict_df['exact'].append(True if f[-1]['max_iou'] > 0.99999999 else False)
                    dict_df['bounded'].append(bounded)
    except NotADirectoryError:
        print("{} is not a directory".format(directory))
        
bounded = True
for directory in bounded_directories:
    # Return all files within directory
    try:
        files = os.listdir(os.path.join(bounded_log, directory))
        for file in files:
            if file.endswith('.json'):
                with open(os.path.join(bounded_log, directory, file), 'r') as f:
                    f = json.load(f)
                    dict_df['uuid'].append(directory)
                    dict_df['used_budget'].append(f[-1]['used_budget'])
                    dict_df['max_iou'].append(f[-1]['max_iou'])
                    dict_df['exact'].append(True if f[-1]['max_iou'] > 0.99999999 else False)
                    dict_df['bounded'].append(bounded)
    except NotADirectoryError:
        print("{} is not a directory".format(directory))

.DS_Store is not a directory
search_results.json is not a directory
.DS_Store is not a directory
search_results.json is not a directory


In [92]:
dict_df

{'uuid': ['54077_dd26efde_0000',
  '33967_0741e236_0002',
  '142764_e4f86a3b_0000',
  '77211_d46ae17d_0013',
  '30246_6e835e6d_0001',
  '41026_295d1dc8_0003',
  '41234_74275eb0_0009',
  '22457_a6c2776f_0008',
  '27839_4a077326_0010',
  '65211_73eab9de_0000',
  '22432_e4a51ee9_0006',
  '54077_dd26efde_0000',
  '33967_0741e236_0002',
  '142764_e4f86a3b_0000',
  '77211_d46ae17d_0013',
  '30246_6e835e6d_0001',
  '41026_295d1dc8_0003',
  '41234_74275eb0_0009',
  '22457_a6c2776f_0008',
  '27839_4a077326_0010',
  '65211_73eab9de_0000',
  '22432_e4a51ee9_0006'],
 'used_budget': [20,
  2,
  1,
  20,
  5,
  1,
  8,
  1,
  1,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20],
 'max_iou': [0.8885589429711583,
  1.0,
  1.0,
  0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9996491787019813,
  0.8846035521300826,
  0.056893078284160514,
  0.1866434059880256,
  0,
  0,
  0.5745608575960637,
  0,
  0.7699510644313011,
  0,
  0,
  0.9436155308507842,
  0.7037559354410343],
 'exact

In [93]:
df = pd.DataFrame(dict_df)

In [94]:
df.groupby(by='bounded')['max_iou'].describe()

,count,mean,std,min,25%,50%,75%,max
bounded,,,,,,,,
False,11.0,0.888437,0.298132,0.0,0.944104,1.000000,1.000000,1.000000
True,11.0,0.294129,0.373506,0.0,0.000000,0.056893,0.639158,0.943616


In [95]:
df.groupby(by='bounded')['used_budget'].describe()

,count,mean,std,min,25%,50%,75%,max
bounded,,,,,,,,
False,11.0,9.0,8.97775,1.0,1.0,5.0,20.0,20.0
True,11.0,20.0,0.00000,20.0,20.0,20.0,20.0,20.0


#### Conclusion

We tested the new search algorithm against the `search` function in `search_best` on a subset of the test set. Testing on the entire set proved to be a time intensive task, and early results from the subset were not promising. 

The implementation did not work as expected. Unfortunately, the agent is not able to use the faces of the bounding box to create extrude cuts, so the best we were able to do was an exact negative of the object. The tables above show the maximum `IoU` achieved for the `bounded=True` group was 0.943, so no exact reconstructions were created. Further the mean `IoU` was only $0.29$ as compared to $0.88$ for the original implementation.

The figures below show the issue. The bounding box is created, but often the only valid moves left are to cut away the target object.

<p float="left"> 
  <img src="images/bounded_search_1.png" style="display:inline;margin:1px;height:300px" /> 
  <img src="images/bounded_searach_2.png" style="display:inline;margin:1px;height:300px" />  
  <img src="images/bounded_search_3.png" style="display:inline;margin:1px;height:300px" /> 
</p>

<center> Search with bounded object and operation space reduced to cuts only </center>

We do believe though that this implementation could work with added layers of complexity that were not available through the original `train` and `search` functionality provided. To replicate this, the process could be something such as the following psuedocode below:

1. `set_target(target)`
2. `create_bounding_box(target)`
3. `inverse_target = intersect_cut_operation() #returns a body that is the volume of the bounding box specifically not within the target`
4. `set_target(inverse_target)`
5. `reconstruct(target) #Until IoU=0`
6. `inverse the steps found in reconstruct #joins become cuts and vice versa`
7. `Insert bounding box as first step`

If this was run in parallel or sequentially with the additive search, we may be able to improve the exact reconstruction percent of the search features. However, we were not able to implement this function with the tools provided